In [ ]:
%%file MyEgaroucid_console.cpp
/*
	Egaroucid Project

	@file Egaroucid_console.cpp
		Main file for Console application
	@date 2021-2023
	@author Takuto Yamana (a.k.a. Nyanyan)
	@license GPL-3.0 license
*/

#include <iostream>
#include "/content/drive/MyDrive/MyIntern/engine/engine_all.hpp"
#include "/content/drive/MyDrive/MyIntern/console/console_all.hpp"

void init_console(Options options){
    thread_pool.resize(std::max(0, options.n_threads - 1));
    bit_init();
    mobility_init();
    flip_init();
    endsearch_init();
    #if USE_MPC_PRE_CALCULATION
        mpc_init();
    #endif
    hash_resize(DEFAULT_HASH_LEVEL, options.hash_level, options.binary_path, options.show_log);
    stability_init();
    if (!evaluate_init(options.eval_file, options.show_log))
        std::exit(0);
    if (!options.nobook)
        book_init(options.book_file, options.show_log);
    if (options.show_log)
        std::cerr << "initialized" << std::endl;
}

int main(int argc, char* argv[]){
    State state;
    std::string binary_path = get_binary_path();
    std::vector<Commandline_option> commandline_options = get_commandline_options(argc, argv);
    Options options = get_options(commandline_options, binary_path);
    print_special_commandline_options(commandline_options);
    init_console(options);
    execute_special_tasks();
    execute_special_commandline_tasks(commandline_options, &options, &state);
    Board_info board;
    board.reset();
    while (true){
        if (options.gtp){
            gtp_check_command(&board, &state, &options);
        }else {
            if (!options.quiet){
                print_board_info(&board);
                std::cout << std::endl;
            }
            if (!execute_special_tasks_loop(&board, &state, &options))
                check_command(&board, &state, &options);
        }
    }
    return 0;
}

Writing MyEgaroucid_console.cpp


In [ ]:
!g++ -O2 MyEgaroucid_console.cpp -o /content/drive/MyDrive/MyIntern/bin/MyEgaroucid_for_Console.out -mtune=native -march=native -mfpmath=both -pthread -std=c++17 -Wall -Wextra -DHAS_NO_AVX2

In file included from /content/drive/MyDrive/MyIntern/engine/midsearch.hpp:17,
                 from /content/drive/MyDrive/MyIntern/engine/ai.hpp:17,
                 from /content/drive/MyDrive/MyIntern/engine/engine_all.hpp:12,
                 from MyEgaroucid_console.cpp:12:
/content/drive/MyDrive/MyIntern/engine/common.hpp: In function ‘int get_localtime(tm*, time_t*)’:
/content/drive/MyDrive/MyIntern/engine/common.hpp:182:27: warning: parameter ‘a’ set but not used [-Wunused-but-set-parameter]
  182 |     int get_localtime(tm* a, time_t* b) {
      |                       ~~~~^
/content/drive/MyDrive/MyIntern/engine/common.hpp: In function ‘void calc_date(int*, int*, int*, int*, int*, int*)’:
/content/drive/MyDrive/MyIntern/engine/common.hpp:221:21: warning: ‘newtime.tm::tm_year’ is used uninitialized in this function [-Wuninitialized]
  221 |     *year = newtime.tm_year + 1900;
      |             ~~~~~~~~^~~~~~~
/content/drive/MyDrive/MyIntern/engine/common.hpp:222:22: warning

In [ ]:
import subprocess

level = "16"# 1~60 0:crash 読む手数
# eval = 

myegaroucid = subprocess.Popen(('/content/drive/MyDrive/MyIntern/bin/MyEgaroucid_for_Console.out -quiet -nobook -level ' + level ).split(), stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)
# + ' -eval ' + eval



In [ ]:
# setboard コマンドで盤面入力 "-","B","W"いずれか64ケタと"B","W"で手番指定
# setboard + " " + ""(64ケタ) + ""(1ケタ)

# setboardコマンド生成
cmd_ln = 'setboard '
for i in range(8):
    for j in range(8):
        if board[i][j] == 'black': #仮
            cmd_ln += 'B'
        elif board[i][j] == 'white': #仮
            cmd_ln += 'W'
        else:
            cmd_ln += '-'
if player == 'black': #仮
    cmd_ln += 'W'
else:
    cmd_ln += 'B'
cmd_ln += '\n'

# setboardコマンド実行

myegaroucid.stdin.write(cmd_ln.encode('utf-8'))
myegaroucid.stdin.flush()
myegaroucid.stdin.write('go\n'.encode('utf-8'))
myegaroucid.stdin.flush()


上記setboard コマンドへの応答として、MyEgaroucid_for_Console.outがA1~H8までの評価関数を出力するように改造することが目標

デフォルトの出力例(現状):

最善手: … A2 …

望ましい出力の変更例:

A1: None

A2: 16

:

H8: -16

以上をPopen.stdout.readline()で回収し、盤面状の配列に戻す

In [ ]:
#書きかけ

eval_result = 

read_cmd_ln = ''
while read_cmd_ln == '':
    read_cmd_ln = myegaroucid.stdout.readline().decode().replace('\n', '')
#eval_result = 







In [ ]:
# MyEgaroucid_for_Console.outの終了

myegaroucid.stdin.write('quit\n'.encode('utf-8'))
myegaroucid.stdin.flush()